# Graph Theory: A Complete Tutorial

This notebook provides a comprehensive introduction to graph theory, covering fundamental concepts, representations, and algorithms with practical Python implementations.

## Table of Contents
1. [What is a Graph?](#what-is-a-graph)
2. [Graph Terminology](#terminology)
3. [Graph Representations](#representations)
4. [Breadth-First Search (BFS)](#bfs)
5. [Depth-First Search (DFS)](#dfs)
6. [Shortest Path Algorithms - SPFA](#spfa)
7. [Practical Examples](#examples)

## 1. What is a Graph? <a id='what-is-a-graph'></a>

A **graph** is a mathematical structure used to model relationships between objects. It consists of:

- **Vertices (or Nodes)**: The objects or entities
- **Edges**: The connections or relationships between vertices

### Real-World Examples:
- **Social Networks**: People (vertices) connected by friendships (edges)
- **Road Maps**: Cities (vertices) connected by roads (edges)
- **Web Pages**: Pages (vertices) connected by hyperlinks (edges)
- **Computer Networks**: Computers (vertices) connected by cables (edges)

### Types of Graphs:

1. **Directed vs Undirected**
   - **Undirected**: Edges have no direction (friendship: if A is friends with B, then B is friends with A)
   - **Directed**: Edges have direction (following on Twitter: A can follow B without B following A)

2. **Weighted vs Unweighted**
   - **Unweighted**: All edges are equal (simple connections)
   - **Weighted**: Edges have values/costs (distances, costs, capacities)

3. **Connected vs Disconnected**
   - **Connected**: There's a path between any two vertices
   - **Disconnected**: Some vertices cannot reach others

## 2. Graph Terminology <a id='terminology'></a>

- **Vertex/Node**: A point in the graph
- **Edge**: A connection between two vertices
- **Degree**: Number of edges connected to a vertex
- **Path**: A sequence of vertices connected by edges
- **Cycle**: A path that starts and ends at the same vertex
- **Adjacent vertices**: Vertices connected by an edge
- **Neighbor**: An adjacent vertex
- **Distance**: Number of edges in the shortest path between two vertices

## 3. Graph Representations <a id='representations'></a>

There are two main ways to represent graphs in computer programs:

### 3.1 Adjacency List

An **adjacency list** stores a list of neighbors for each vertex. This is memory-efficient for sparse graphs (graphs with relatively few edges).

**Structure**: A dictionary where:
- Keys = vertices
- Values = lists of neighboring vertices

**Example**: For an undirected graph with edges (0-1), (0-2), (1-3)
```python
graph = {
    0: [1, 2],
    1: [0, 3],
    2: [0],
    3: [1]
}
```

**Pros**:
- Space efficient: O(V + E) where V = vertices, E = edges
- Fast to iterate through neighbors
- Good for sparse graphs

**Cons**:
- Slower to check if an edge exists between two specific vertices

In [ ]:
# Adjacency List Implementation

class GraphList:
    def __init__(self):
        self.adjacency_list = {}
    
    def add_vertex(self, vertex):
        """Add a new vertex to the graph"""
        if vertex not in self.adjacency_list:
            self.adjacency_list[vertex] = []
    
    def add_edge(self, u, v, directed=False):
        """Add an edge between vertices u and v"""
        # Ensure both vertices exist
        self.add_vertex(u)
        self.add_vertex(v)
        
        # Add edge u -> v
        self.adjacency_list[u].append(v)
        
        # For undirected graph, also add v -> u
        if not directed:
            self.adjacency_list[v].append(u)
    
    def display(self):
        """Display the graph"""
        print("Graph (Adjacency List):")
        for vertex, neighbors in self.adjacency_list.items():
            print(f"  {vertex} -> {neighbors}")

# Example usage
g = GraphList()
g.add_edge(0, 1)
g.add_edge(0, 2)
g.add_edge(1, 3)
g.add_edge(2, 3)
g.display()

### 3.2 Adjacency Matrix

An **adjacency matrix** is a 2D array where:
- Rows and columns represent vertices
- `matrix[i][j] = 1` if there's an edge from vertex i to vertex j
- `matrix[i][j] = 0` if there's no edge

**Example**: For the same graph as above (4 vertices)
```
    0  1  2  3
0 [ 0  1  1  0 ]
1 [ 1  0  0  1 ]
2 [ 1  0  0  1 ]
3 [ 0  1  1  0 ]
```

**Pros**:
- Fast edge lookup: O(1) to check if edge exists
- Simple to implement
- Good for dense graphs

**Cons**:
- Space inefficient: Always uses O(V²) space
- Slower to iterate through all neighbors

In [ ]:
# Adjacency Matrix Implementation

class GraphMatrix:
    def __init__(self, num_vertices):
        self.num_vertices = num_vertices
        self.matrix = [[0] * num_vertices for _ in range(num_vertices)]
    
    def add_edge(self, u, v, directed=False):
        """Add an edge between vertices u and v"""
        if u >= self.num_vertices or v >= self.num_vertices:
            raise ValueError("Vertex index out of bounds")
        
        self.matrix[u][v] = 1
        if not directed:
            self.matrix[v][u] = 1
    
    def display(self):
        """Display the adjacency matrix"""
        print("Graph (Adjacency Matrix):")
        print("  ", " ".join(str(i) for i in range(self.num_vertices)))
        for i, row in enumerate(self.matrix):
            print(f"{i} [{' '.join(str(x) for x in row)}]")

# Example usage
gm = GraphMatrix(4)
gm.add_edge(0, 1)
gm.add_edge(0, 2)
gm.add_edge(1, 3)
gm.add_edge(2, 3)
gm.display()

### Comparison: When to Use Which?

| Criterion | Adjacency List | Adjacency Matrix |
|-----------|---------------|------------------|
| **Space** | O(V + E) | O(V²) |
| **Check if edge exists** | O(degree) | O(1) |
| **Iterate neighbors** | O(degree) | O(V) |
| **Best for** | Sparse graphs | Dense graphs |
| **Add vertex** | Easy | Expensive (resize) |

**Rule of thumb**: Use adjacency list for most real-world graphs (social networks, road maps, etc.)

## 4. Breadth-First Search (BFS) <a id='bfs'></a>

**BFS** explores a graph level by level, starting from a source vertex. It visits all neighbors before moving to the next level.

### How BFS Works:

1. Start at a source vertex and mark it as visited
2. Add it to a queue
3. While the queue is not empty:
   - Remove the first vertex from the queue
   - Visit all its unvisited neighbors
   - Mark them as visited and add them to the queue

### Visual Example:

```
Graph:     0 --- 1
           |     |
           2 --- 3 --- 4
```

BFS from vertex 0:
- Level 0: Visit 0
- Level 1: Visit 1, 2 (neighbors of 0)
- Level 2: Visit 3 (neighbor of 1 and 2)
- Level 3: Visit 4 (neighbor of 3)

**Order**: [0, 1, 2, 3, 4]

### Applications:
- Finding shortest path in unweighted graphs
- Level-order traversal
- Finding connected components
- Web crawling

### Time Complexity: O(V + E)
### Space Complexity: O(V)

In [ ]:
from collections import deque
from typing import Dict, List, Set

def bfs(graph: Dict[int, List[int]], start: int) -> List[int]:
    """
    Perform Breadth-First Search starting from a given vertex.
    
    Args:
        graph: Adjacency list representation
        start: Starting vertex
    
    Returns:
        List of vertices in BFS visitation order
    """
    visited = set()
    order = []
    queue = deque([start])
    visited.add(start)
    
    while queue:
        current = queue.popleft()  # Remove from front of queue
        order.append(current)
        
        # Visit all unvisited neighbors
        for neighbor in graph.get(current, []):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)  # Add to back of queue
    
    return order

# Example graph
graph = {
    0: [1, 2],
    1: [0, 3],
    2: [0, 3],
    3: [1, 2, 4],
    4: [3]
}

print("Graph:", graph)
print("\nBFS starting from vertex 0:", bfs(graph, 0))
print("BFS starting from vertex 2:", bfs(graph, 2))

### BFS Step-by-Step Example

Let's trace through BFS on a simple graph:

In [ ]:
def bfs_detailed(graph: Dict[int, List[int]], start: int):
    """
    BFS with detailed step-by-step output for learning.
    """
    visited = set()
    order = []
    queue = deque([start])
    visited.add(start)
    
    print(f"Starting BFS from vertex {start}\n")
    step = 1
    
    while queue:
        print(f"Step {step}:")
        print(f"  Queue: {list(queue)}")
        print(f"  Visited so far: {sorted(visited)}")
        
        current = queue.popleft()
        order.append(current)
        print(f"  Processing vertex: {current}")
        
        neighbors = graph.get(current, [])
        print(f"  Neighbors of {current}: {neighbors}")
        
        for neighbor in neighbors:
            if neighbor not in visited:
                print(f"    -> Adding {neighbor} to queue")
                visited.add(neighbor)
                queue.append(neighbor)
        
        print()
        step += 1
    
    print(f"Final BFS order: {order}")
    return order

# Small example graph
simple_graph = {
    0: [1, 2],
    1: [3],
    2: [3],
    3: []
}

bfs_detailed(simple_graph, 0)

## 5. Depth-First Search (DFS) <a id='dfs'></a>

**DFS** explores a graph by going as deep as possible along each branch before backtracking.

### How DFS Works:

1. Start at a source vertex and mark it as visited
2. For each unvisited neighbor:
   - Recursively visit that neighbor (go deep)
3. Backtrack when no unvisited neighbors remain

### Visual Example:

```
Graph:     0 --- 1
           |     |
           2 --- 3 --- 4
```

DFS from vertex 0 (exploring left neighbor first):
- Visit 0
- Visit 1 (first neighbor of 0)
- Visit 3 (first neighbor of 1)
- Visit 2 (first neighbor of 3, not visited yet)
- Visit 4 (backtrack to 3, then visit 4)

**Possible order**: [0, 1, 3, 2, 4] or [0, 1, 3, 4, 2] (depends on neighbor order)

### Comparison: BFS vs DFS

| Aspect | BFS | DFS |
|--------|-----|-----|
| **Data Structure** | Queue (FIFO) | Stack/Recursion (LIFO) |
| **Explores** | Level by level | Deep first |
| **Best for** | Shortest path | Connectivity, cycles |
| **Memory** | Can use more | Less for deep graphs |

### Applications:
- Finding connected components
- Detecting cycles
- Topological sorting
- Maze solving
- Path finding

### Time Complexity: O(V + E)
### Space Complexity: O(V)

In [ ]:
def dfs_recursive(graph: Dict[int, List[int]], start: int, visited: Set[int] = None) -> Set[int]:
    """
    Perform Depth-First Search using recursion.
    
    Args:
        graph: Adjacency list representation
        start: Starting vertex
        visited: Set of visited vertices (for recursion)
    
    Returns:
        Set of all visited vertices
    """
    if visited is None:
        visited = set()
    
    visited.add(start)
    
    for neighbor in graph.get(start, []):
        if neighbor not in visited:
            dfs_recursive(graph, neighbor, visited)
    
    return visited

# Same example graph as BFS
graph = {
    0: [1, 2],
    1: [0, 3],
    2: [0, 3],
    3: [1, 2, 4],
    4: [3]
}

print("Graph:", graph)
print("\nDFS starting from vertex 0:", sorted(dfs_recursive(graph, 0)))
print("DFS starting from vertex 2:", sorted(dfs_recursive(graph, 2)))

In [ ]:
def dfs_iterative(graph: Dict[int, List[int]], start: int) -> List[int]:
    """
    Perform DFS using an explicit stack (iterative approach).
    Returns vertices in the order they were visited.
    """
    visited = set()
    order = []
    stack = [start]
    
    while stack:
        current = stack.pop()  # Remove from top of stack
        
        if current not in visited:
            visited.add(current)
            order.append(current)
            
            # Add neighbors to stack (in reverse order to match recursive DFS)
            for neighbor in reversed(graph.get(current, [])):
                if neighbor not in visited:
                    stack.append(neighbor)
    
    return order

print("\nDFS (Iterative) starting from vertex 0:", dfs_iterative(graph, 0))

### DFS Step-by-Step Example

In [ ]:
def dfs_detailed(graph: Dict[int, List[int]], start: int, visited: Set[int] = None, depth: int = 0) -> Set[int]:
    """
    DFS with detailed step-by-step output for learning.
    """
    if visited is None:
        visited = set()
        print(f"Starting DFS from vertex {start}\n")
    
    indent = "  " * depth
    print(f"{indent}Visiting vertex: {start}")
    visited.add(start)
    
    neighbors = graph.get(start, [])
    print(f"{indent}Neighbors of {start}: {neighbors}")
    
    for neighbor in neighbors:
        if neighbor not in visited:
            print(f"{indent}-> Going deep to {neighbor}")
            dfs_detailed(graph, neighbor, visited, depth + 1)
            print(f"{indent}<- Backtracking from {neighbor} to {start}")
        else:
            print(f"{indent}  (Skipping {neighbor}, already visited)")
    
    return visited

# Small example graph
simple_graph = {
    0: [1, 2],
    1: [3],
    2: [3],
    3: []
}

result = dfs_detailed(simple_graph, 0)
print(f"\nFinal visited vertices: {sorted(result)}")

## 6. Shortest Path Algorithms - SPFA <a id='spfa'></a>

### The Shortest Path Problem

Given a **weighted graph**, find the shortest path from a source vertex to all other vertices.

### SPFA: Shortest Path Faster Algorithm

SPFA is an improvement over the Bellman-Ford algorithm. It uses a queue to track vertices that need to be processed.

### How SPFA Works:

1. Initialize distances: source = 0, all others = ∞
2. Add source to queue
3. While queue is not empty:
   - Remove a vertex u from queue
   - For each neighbor v of u:
     - If distance[u] + weight(u,v) < distance[v]:
       - Update distance[v]
       - Add v to queue (if not already in queue)
4. Detect negative cycles (if a vertex is relaxed too many times)

### Key Concepts:

**Relaxation**: Updating the distance to a vertex if a shorter path is found.

```
if distance[u] + weight(u,v) < distance[v]:
    distance[v] = distance[u] + weight(u,v)
```

**Negative Cycle**: A cycle whose total edge weight is negative. If present, there's no shortest path (you can keep going around the cycle to get infinitely short paths).

### Example:

```
Graph:  0 --(5)--> 1
        |          |
       (4)       (3)
        |          |
        v          v
        2 --(6)--> 3
```

Shortest paths from 0:
- 0 → 0: 0
- 0 → 1: 5
- 0 → 2: 4
- 0 → 3: 8 (via 0→1→3)

### Time Complexity:
- Average case: O(E) - very fast in practice
- Worst case: O(V × E) - same as Bellman-Ford

### Comparison with Other Algorithms:

| Algorithm | Time | Negative Weights? | Use Case |
|-----------|------|-------------------|----------|
| **Dijkstra** | O(E log V) | ❌ No | Fastest for non-negative |
| **Bellman-Ford** | O(V × E) | ✅ Yes | Guaranteed, but slow |
| **SPFA** | O(E) avg | ✅ Yes | Fast in practice |

In [ ]:
from collections import deque
from typing import Dict, List, Tuple

def spfa(graph: Dict[int, List[Tuple[int, float]]], start: int) -> Dict[int, float]:
    """
    Shortest Path Faster Algorithm (SPFA).
    
    Args:
        graph: Adjacency list with weights. Format: {node: [(neighbor, weight), ...]}
        start: Starting vertex
    
    Returns:
        Dictionary of shortest distances from start to each vertex
    
    Raises:
        ValueError: If negative weight cycle is detected
    """
    # Collect all vertices
    vertices = set(graph.keys())
    for u in graph:
        for v, _ in graph[u]:
            vertices.add(v)
    
    # Initialize distances
    distances = {v: float('inf') for v in vertices}
    distances[start] = 0
    
    # Queue for processing
    queue = deque([start])
    in_queue = {v: False for v in vertices}
    in_queue[start] = True
    
    # Count relaxations to detect negative cycles
    relax_count = {v: 0 for v in vertices}
    
    while queue:
        u = queue.popleft()
        in_queue[u] = False
        
        for v, weight in graph.get(u, []):
            # Relaxation step
            if distances[u] + weight < distances[v]:
                distances[v] = distances[u] + weight
                relax_count[v] += 1
                
                # Negative cycle detection
                if relax_count[v] > len(vertices):
                    raise ValueError("Negative weight cycle detected")
                
                if not in_queue[v]:
                    queue.append(v)
                    in_queue[v] = True
    
    return distances

# Example: Graph with positive weights
weighted_graph = {
    0: [(1, 5.0), (2, 4.0)],
    1: [(3, 3.0)],
    2: [(1, 2.0), (3, 6.0)],
    3: []
}

print("Graph:", weighted_graph)
print("\nShortest distances from vertex 0:")
distances = spfa(weighted_graph, 0)
for vertex, distance in sorted(distances.items()):
    print(f"  To vertex {vertex}: {distance}")

### SPFA with Negative Weights Example

In [ ]:
# Example with negative weights (but no negative cycle)
graph_negative = {
    0: [(1, 4.0), (2, 3.0)],
    1: [(3, -2.0)],  # Negative weight!
    2: [(3, 5.0)],
    3: []
}

print("Graph with negative weights:", graph_negative)
print("\nShortest distances from vertex 0:")
try:
    distances = spfa(graph_negative, 0)
    for vertex, distance in sorted(distances.items()):
        print(f"  To vertex {vertex}: {distance}")
except ValueError as e:
    print(f"Error: {e}")

### Example: Detecting Negative Cycles

In [ ]:
# Example with a negative cycle
graph_with_cycle = {
    0: [(1, 5.0)],
    1: [(2, 3.0)],
    2: [(3, -2.0)],
    3: [(1, -8.0)]  # Creates negative cycle: 1 -> 2 -> 3 -> 1 (total: 3 + (-2) + (-8) = -7)
}

print("Graph with negative cycle:", graph_with_cycle)
print("\nTrying to find shortest paths from vertex 0:")
try:
    distances = spfa(graph_with_cycle, 0)
    for vertex, distance in sorted(distances.items()):
        print(f"  To vertex {vertex}: {distance}")
except ValueError as e:
    print(f"❌ Error: {e}")
    print("\nExplanation: The cycle 1 → 2 → 3 → 1 has total weight -7.")
    print("Going around this cycle repeatedly makes distances infinitely small!")

## 7. Practical Examples <a id='examples'></a>

Let's apply what we've learned to solve real-world problems.

### Example 1: Social Network - Finding Degrees of Connection

Problem: In a social network, find the minimum number of connections between two people.

In [ ]:
def degrees_of_separation(graph: Dict[str, List[str]], person1: str, person2: str) -> int:
    """
    Find the minimum degrees of separation between two people.
    Uses BFS to find shortest path.
    """
    if person1 == person2:
        return 0
    
    visited = {person1}
    queue = deque([(person1, 0)])  # (person, distance)
    
    while queue:
        current, distance = queue.popleft()
        
        for friend in graph.get(current, []):
            if friend == person2:
                return distance + 1
            
            if friend not in visited:
                visited.add(friend)
                queue.append((friend, distance + 1))
    
    return -1  # Not connected

# Social network example
social_network = {
    'Alice': ['Bob', 'Carol'],
    'Bob': ['Alice', 'David', 'Eve'],
    'Carol': ['Alice', 'Frank'],
    'David': ['Bob'],
    'Eve': ['Bob', 'Frank'],
    'Frank': ['Carol', 'Eve', 'Grace'],
    'Grace': ['Frank']
}

print("Social Network:")
for person, friends in social_network.items():
    print(f"  {person} is friends with: {', '.join(friends)}")

print("\nDegrees of separation:")
print(f"  Alice to David: {degrees_of_separation(social_network, 'Alice', 'David')} degrees")
print(f"  Alice to Grace: {degrees_of_separation(social_network, 'Alice', 'Grace')} degrees")
print(f"  Bob to Frank: {degrees_of_separation(social_network, 'Bob', 'Frank')} degrees")

### Example 2: City Navigation - Finding Cheapest Route

Problem: Find the cheapest route between cities.

In [ ]:
# City transportation network with costs
city_network = {
    'NYC': [('Boston', 50), ('Philadelphia', 30)],
    'Boston': [('NYC', 50), ('Portland', 80)],
    'Philadelphia': [('NYC', 30), ('Washington', 40), ('Pittsburgh', 100)],
    'Washington': [('Philadelphia', 40), ('Richmond', 60)],
    'Pittsburgh': [('Philadelphia', 100), ('Cleveland', 70)],
    'Cleveland': [('Pittsburgh', 70)],
    'Portland': [('Boston', 80)],
    'Richmond': [('Washington', 60)]
}

print("City Network:")
for city, routes in city_network.items():
    print(f"  From {city}: {routes}")

print("\nCheapest routes from NYC:")
try:
    costs = spfa(city_network, 'NYC')
    for city, cost in sorted(costs.items()):
        if cost == float('inf'):
            print(f"  To {city}: Not reachable")
        else:
            print(f"  To {city}: ${cost}")
except ValueError as e:
    print(f"Error: {e}")

### Example 3: Connected Components

Problem: Find all separate groups in a network (e.g., friend circles, isolated networks).

In [ ]:
def find_connected_components(graph: Dict[int, List[int]]) -> List[Set[int]]:
    """
    Find all connected components in an undirected graph.
    Uses DFS to explore each component.
    """
    visited = set()
    components = []
    
    # Get all vertices
    all_vertices = set(graph.keys())
    for neighbors in graph.values():
        all_vertices.update(neighbors)
    
    for vertex in all_vertices:
        if vertex not in visited:
            # Start a new component
            component = set()
            
            # DFS to find all vertices in this component
            stack = [vertex]
            while stack:
                current = stack.pop()
                if current not in visited:
                    visited.add(current)
                    component.add(current)
                    stack.extend(graph.get(current, []))
            
            components.append(component)
    
    return components

# Graph with multiple disconnected components
disconnected_graph = {
    0: [1, 2],
    1: [0],
    2: [0],
    3: [4, 5],
    4: [3],
    5: [3],
    6: [7],
    7: [6],
    8: []  # Isolated vertex
}

print("Graph:", disconnected_graph)
print("\nConnected components:")
components = find_connected_components(disconnected_graph)
for i, component in enumerate(components, 1):
    print(f"  Component {i}: {sorted(component)}")

## Summary

### Key Takeaways:

1. **Graphs** model relationships: vertices (objects) connected by edges (relationships)

2. **Representations**:
   - Adjacency List: Memory-efficient, good for sparse graphs
   - Adjacency Matrix: Fast edge lookup, good for dense graphs

3. **Traversal Algorithms**:
   - **BFS**: Level by level, shortest path in unweighted graphs
   - **DFS**: Deep first, good for connectivity and cycles

4. **Shortest Path**:
   - **SPFA**: Efficient for weighted graphs, handles negative weights
   - Detects negative cycles

### When to Use What?

- **Need shortest path (unweighted)?** → BFS
- **Need to explore all vertices?** → DFS or BFS
- **Need shortest path (weighted)?** → SPFA or Dijkstra
- **Have negative weights?** → SPFA or Bellman-Ford
- **Need to find cycles?** → DFS
- **Need to find connected components?** → DFS

### Practice Tips:

1. Start with small graphs and trace through algorithms manually
2. Draw graphs on paper to visualize the problem
3. Implement the basic algorithms from scratch
4. Solve problems on LeetCode, HackerRank, or Codeforces
5. Think about real-world applications

## Further Reading and Resources

### Books:
- "Introduction to Algorithms" by Cormen, Leiserson, Rivest, and Stein (CLRS)
- "Algorithms" by Robert Sedgewick and Kevin Wayne
- "Competitive Programmer's Handbook" by Antti Laaksonen

### Online Resources:
- [Visualgo](https://visualgo.net/) - Algorithm visualizations
- [Graph Theory by William Fiset (YouTube)](https://www.youtube.com/watch?v=09_LlHjoEiY)
- [LeetCode Graph Problems](https://leetcode.com/tag/graph/)

### Practice Problems:
1. Find if path exists between two vertices
2. Count number of islands (2D grid)
3. Course schedule (topological sort)
4. Network delay time (shortest path)
5. Minimum spanning tree problems